In [1]:
import json
import pandas as pd

filename = 'test.jsonl'
savename = 'test.csv'

data = []
with open(filename) as f:
    for line in f:
        data.append(json.loads(line))

data2 = []
for i in data:
    data2.append(i['text'])

data2 = pd.DataFrame(data2, columns=['text'])

labels = []
for i in data:
    labels.append(i['label'])
data2['label'] = labels
data2.to_csv(savename, index=False)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3058682447.py, line 4)

In [25]:
import torch
import torch.nn as nn
from torch import Tensor
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [26]:
class SampleLoader(Dataset):
    def __init__(self, annotation_file):
        super().__init__()
        self.samples = pd.read_csv(annotation_file)
        
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        txt = self.samples.iloc[idx]['text']
        label = self.samples.iloc[idx]['label']
        return txt, label

In [27]:
trainer = SampleLoader(annotation_file='fake news.csv')
trainloader = DataLoader(dataset=trainer, batch_size=128, shuffle=False)

In [29]:
for i , (text, label) in enumerate(trainloader):
    print(text)
    print(label)
    break

('posted by Eddie Gun man have stolen a Dirty Bomb by force in Mexico and now the bomb can be on its way into the United States. Why is the media remaining silent on such an issue? I do not have any idea, but i know that mainstream media is controlled by 6 giant corporations….. so it raises my curiosity even more. A device containing radioactive substances including cesium-37 and americium-beryllium, was stolen from a government research facility in Mexico. Authorities are currently working to track down the group of men who grabbed the device Sunday night during an assault on a National Construction Laboratory warehouse in Tultitlan, Mexico state.', 'President Donald Trump\'s 2018 budget asks Congress to construct 60 miles of new border walls along the U. S. border — all of which would be located in Texas. [The plan is for the Rio Grande Valley Sector to receive 32 miles of a new border wall, and 28 miles of a new levee wall. The levee in Hidalgo County is designed to address flooding

In [15]:
import pandas as pd

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
val_data = pd.read_csv('validation.csv')

# Inspect the data
print(train_data.head())
print(train_data.columns)

                                                text  label
0  Already, the mayor of Green Bay is having ribb...      1
1  I argued for years that we need to move from a...      1
2  The video doesn't show the broader picture of ...      0
3  A ham and cheese sandwich on one slice of brea...      0
4  He has been able to attract independents and D...      1
Index(['text', 'label'], dtype='object')


In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
val_data = pd.read_csv('validation.csv')

# Define constants
MAX_NUM_WORDS = 10000  # Use the top 10,000 most frequent words
MAX_SEQUENCE_LENGTH = 200  # Maximum length of sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_data['text'].values)

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_data['text'].values)
X_val = tokenizer.texts_to_sequences(val_data['text'].values)
X_test = tokenizer.texts_to_sequences(test_data['text'].values)

# Pad sequences to ensure uniform input size
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# Convert labels to numpy arrays
y_train = train_data['label'].values
y_val = val_data['label'].values
y_test = test_data['label'].values


model = Sequential()
model.add(Embedding(input_dim=MAX_NUM_WORDS, output_dim=128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

C:\Users\Sina\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 32ms/step - accuracy: 0.9621 - loss: 0.1820 - val_accuracy: 0.9580 - val_loss: 0.1339
Epoch 2/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - accuracy: 0.9622 - loss: 0.1053 - val_accuracy: 0.9706 - val_loss: 0.1057
Epoch 3/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.9852 - loss: 0.0457 - val_accuracy: 0.9734 - val_loss: 0.1108
Epoch 4/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - accuracy: 0.9971 - loss: 0.0122 - val_accuracy: 0.9755 - val_loss: 0.1439
Epoch 5/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - accuracy: 0.9989 - loss: 0.0046 - val_accuracy: 0.9706 - val_loss: 0.1829
Epoch 6/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step - accuracy: 0.9997 - loss: 0.0030 - val_accuracy: 0.9706 - val_loss: 0.2354
Epoch 7/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 12s 34ms/step - accuracy: 0.9997 - loss: 0.0029 - val_accuracy: 0.9552 - val_loss: 0.2189
Epoch 8/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.9999 - loss: 0.0012 - 

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
import numpy as np

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
val_data = pd.read_csv('validation.csv')

# Define constants
MAX_NUM_WORDS = 10000  # Use the top 10,000 most frequent words
MAX_SEQUENCE_LENGTH = 200  # Maximum length of sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_data['text'].values)

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_data['text'].values)
X_val = tokenizer.texts_to_sequences(val_data['text'].values)
X_test = tokenizer.texts_to_sequences(test_data['text'].values)

# Pad sequences to ensure uniform input size
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# Convert labels to numpy arrays
y_train = train_data['label'].values
y_val = val_data['label'].values
y_test = test_data['label'].values

# Load GloVe embeddings
def load_glove_embeddings(glove_file, word_index, embedding_dim=100):
    embeddings_index = {}
    with open(glove_file, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

# Path to the GloVe file
glove_file = 'glove.6B.100d.txt'  # Change this to the actual path of the GloVe file
embedding_dim = 100
word_index = tokenizer.word_index
embedding_matrix = load_glove_embeddings(glove_file, word_index, embedding_dim)

# Define the CNN model with pre-trained embeddings and dropout layers
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=MAX_SEQUENCE_LENGTH,
                    trainable=False))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Dropout(0.5))  # Add dropout layer
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Add dropout layer
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')


C:\Users\Sina\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     1,228,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,228,400 (4.69 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,228,400 (4.69 MB)

Epoch 1/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.6274 - loss: 0.6657 - val_accuracy: 0.6464 - val_loss: 0.6366
Epoch 2/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.6504 - loss: 0.6368 - val_accuracy: 0.6440 - val_loss: 0.6294
Epoch 3/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.6503 - loss: 0.6258 - val_accuracy: 0.6448 - val_loss: 0.6280
Epoch 4/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.6557 - loss: 0.6144 - val_accuracy: 0.6488 - val_loss: 0.6251
Epoch 5/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.6604 - loss: 0.6092 - val_accuracy: 0.6288 - val_loss: 0.6286
Epoch 6/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.6810 - loss: 0.5892 - val_accuracy: 0.6376 - val_loss: 0.6203
Epoch 7/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.6978 - loss: 0.5731 - val_accuracy: 0.6424 - val_loss: 0.6206
Epoch 8/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.7110 - loss: 0.5501 - val_accu

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Bidirectional, GlobalMaxPooling1D, Dense, Dropout
import numpy as np
from sklearn.model_selection import KFold

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
val_data = pd.read_csv('validation.csv')

# Combine train and validation data for cross-validation
combined_data = pd.concat([train_data, val_data])

# Define constants
MAX_NUM_WORDS = 10000  # Use the top 10,000 most frequent words
MAX_SEQUENCE_LENGTH = 200  # Maximum length of sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(combined_data['text'].values)

# Convert text to sequences
X = tokenizer.texts_to_sequences(combined_data['text'].values)
X_test = tokenizer.texts_to_sequences(test_data['text'].values)

# Pad sequences to ensure uniform input size
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# Convert labels to numpy arrays
y = combined_data['label'].values
y_test = test_data['label'].values

# Load GloVe embeddings
def load_glove_embeddings(glove_file, word_index, embedding_dim=100):
    embeddings_index = {}
    with open(glove_file, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

# Path to the GloVe file
glove_file = 'glove.6B.100d.txt'  # Change this to the actual path of the GloVe file
embedding_dim = 100
word_index = tokenizer.word_index
embedding_matrix = load_glove_embeddings(glove_file, word_index, embedding_dim)

# Function to create the Bidirectional LSTM model
def create_bilstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=len(word_index) + 1,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH,
                        trainable=False))
    model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Perform K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = create_bilstm_model()
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
    
    loss, accuracy = model.evaluate(X_val, y_val)
    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print(f'Average Cross-Validation Accuracy: {average_accuracy*100:.2f}%')

# Evaluate the final model on the test set
model = create_bilstm_model()
model.fit(X, y, epochs=10, batch_size=32)
loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy*100:.2f}%')


Epoch 1/10


C:\Users\Sina\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


461/461 ━━━━━━━━━━━━━━━━━━━━ 72s 149ms/step - accuracy: 0.7620 - loss: 0.4646 - val_accuracy: 0.8957 - val_loss: 0.2469
Epoch 2/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 70s 152ms/step - accuracy: 0.9165 - loss: 0.2151 - val_accuracy: 0.9272 - val_loss: 0.1855
Epoch 3/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 75s 162ms/step - accuracy: 0.9389 - loss: 0.1559 - val_accuracy: 0.9324 - val_loss: 0.1711
Epoch 4/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 78s 170ms/step - accuracy: 0.9562 - loss: 0.1095 - val_accuracy: 0.9400 - val_loss: 0.1546
Epoch 5/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 82s 177ms/step - accuracy: 0.9712 - loss: 0.0822 - val_accuracy: 0.9435 - val_loss: 0.1569
Epoch 6/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 86s 186ms/step - accuracy: 0.9831 - loss: 0.0481 - val_accuracy: 0.9413 - val_loss: 0.1587
Epoch 7/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 78s 170ms/step - accuracy: 0.9920 - loss: 0.0258 - val_accuracy: 0.8881 - val_loss: 0.4285
Epoch 8/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 81s 176ms/step - accuracy: 0.9885 - loss: 0.0337 - val

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Load the datasets
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('validation.csv')
test_data = pd.read_csv('test.csv')

# Define constants
MAX_NUM_WORDS = 10000  # Use the top 10,000 most frequent words
MAX_SEQUENCE_LENGTH = 200  # Maximum length of sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_data['text'].values)

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_data['text'].values)
X_val = tokenizer.texts_to_sequences(val_data['text'].values)
X_test = tokenizer.texts_to_sequences(test_data['text'].values)

# Pad sequences to ensure uniform input size
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# Convert labels to numpy arrays
y_train = train_data['label'].values
y_val = val_data['label'].values
y_test = test_data['label'].values

# Define the RNN model
model = Sequential()
model.add(Embedding(input_dim=MAX_NUM_WORDS, output_dim=128, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

C:\Users\Sina\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 85s 225ms/step - accuracy: 0.9464 - loss: 0.2177 - val_accuracy: 0.9678 - val_loss: 0.1222
Epoch 2/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 86s 241ms/step - accuracy: 0.9691 - loss: 0.1044 - val_accuracy: 0.9776 - val_loss: 0.0993
Epoch 3/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 85s 238ms/step - accuracy: 0.9854 - loss: 0.0473 - val_accuracy: 0.9734 - val_loss: 0.1160
Epoch 4/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 96s 269ms/step - accuracy: 0.9947 - loss: 0.0205 - val_accuracy: 0.9727 - val_loss: 0.1280
Epoch 5/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 93s 259ms/step - accuracy: 0.9963 - loss: 0.0125 - val_accuracy: 0.9643 - val_loss: 0.1433
Epoch 6/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 88s 246ms/step - accuracy: 0.9979 - loss: 0.0080 - val_accuracy: 0.9741 - val_loss: 0.1693
Epoch 7/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 89s 248ms/step - accuracy: 0.9988 - loss: 0.0058 - val_accuracy: 0.9769 - val_loss: 0.1651
Epoch 8/10
358/358 ━━━━━━━━━━━━━━━━━━━━ 85s 236ms/step - accuracy: 0.9972 - loss: 0